# Always calling a Tool First

In [3]:
import ast
from typing import Annotated, TypedDict
from uuid import uuid4

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import AIMessage, HumanMessage, ToolCall
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI

from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

import os
import sys

# Go up one level from "notebooks/" to project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from utils.display_graph import graph_to_image

@tool
def calculator(query: str) -> str:
    """A simple calculator tool. Input should be a mathematical expression."""
    return ast.literal_eval(query)

search = DuckDuckGoSearchRun()
tools = [search, calculator]
model = AzureChatOpenAI(azure_deployment="gpt-4o",
                        api_version="2024-10-21",
                        model="gpt-4o" #for tracing
                        ).bind_tools(tools) #bind tools as using bind method is just for one tool

class State(TypedDict):
    messages: Annotated[list, add_messages]

def model_node(state: State) -> State:
    res = model.invoke(state["messages"])
    return {"messages":res}

def first_model(state: State) -> State:
    query = state["messages"][-1].content
    search_tool_call = ToolCall(
        name="duckduckgo_search", args={"query":query}, id=uuid4().hex
    )
    return {"messages": AIMessage(content="", tool_calls=[search_tool_call])}

builder = StateGraph(State)
builder.add_node("first_model",first_model)
builder.add_node("model", model_node)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "first_model")
builder.add_edge("first_model", "tools")
builder.add_conditional_edges("model",tools_condition)
builder.add_edge("tools", "model")

graph = builder.compile()

In [4]:
graph_to_image(graph, "graph_rag_for_tool_choosing.png")

'File graph_rag_for_tool_choosing.png already exists. Skipping image generation.'

![Tool first agent](../img/graph_tool_first.png)


This version of the agent architecture introduces a small but impactful change in how execution begins:

---

### ✅ Step 1: `first_model` Node

- This node **does not call an LLM**.
- Instead, it **automatically generates a tool call** to the `search` tool.
- It uses the **user's message verbatim** as the search query.

In contrast, the previous architecture relied on the **LLM to decide** whether to call the search tool or not.

---

### 🔧 Step 2: `tools` Node

- This node is **identical** to the previous architecture.
- It executes the `search` tool and returns the results.

---

### 🤖 Step 3: `agent` Node

- Now the system enters the LLM loop.
- From this point forward, the LLM **decides what to do next**, based on the results of the initial tool execution.

---

### 🎯 Why This Matters

By initiating the search manually:
- You reduce **latency** by skipping the first LLM decision step.
- You ensure that **search always runs first**, increasing consistency.
- You eliminate the risk of the LLM **skipping a crucial tool call**.

---

### 📤 Example Output

Next, let’s take a look at the output for the same query used earlier, now following this updated execution flow.

In [ ]:
input = {
    "messages": [
        HumanMessage("""How old was the 30th president of the United States 
            when he died?""")
    ]
}
for c in graph.stream(input):
    print(c)

{'first_model': {'messages': AIMessage(content='', additional_kwargs={}, response_metadata={}, id='0ed645d0-63bc-4254-bab2-36b3999b2c1c', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'How old was the 30th president of the United States \n            when he died?'}, 'id': '7c6daa8d0f684f4dbc2092015b04c592', 'type': 'tool_call'}])}}
{'tools': {'messages': [ToolMessage(content='Calvin Coolidge (born John Calvin Coolidge Jr.; [1] / ˈ k uː l ɪ dʒ / KOOL-ij; July 4, 1872 - January 5, 1933) was the 30th president of the United States, serving from 1923 to 1929.A Republican lawyer from Massachusetts, he previously served as the 29th vice president from 1921 to 1923 under President Warren G. Harding, and as the 48th governor of Massachusetts from 1919 to 1921. Calvin Coolidge was the 30th president of the United States (1923-29). Coolidge acceded to the presidency after the death in office of Warren G. Harding, just as the Harding scandals were coming to light. ... 1872, Plymout

Note how this time, we skipped the initial LLM call. Used instead *first model* node which directly returned a tool call for the search tool. From there the same process was used as in the previous flow. We first executed the search tool and finally went back to the **model** node to generate the final anwser.

Let's see another usage

## Dealing with Many Tools
Large Language Models (LLMs) are powerful—but not flawless. One known limitation is their difficulty with **handling too many options** or **overly verbose prompts**.

This issue is particularly evident when it comes to **planning the next tool to use**.

---

### ⚠️ Problem: Too Many Tools

When presented with **more than 10 tools**, an LLM’s ability to:
- Select the correct tool
- Plan actions effectively

...begins to **decline noticeably**.

---

### ✅ Solution: Use a RAG-Based Tool Selector

If your application relies on **many specialized tools**, a better approach is to:
1. Use a **Retrieval-Augmented Generation (RAG)** step to:
   - Preselect the most relevant tools based on the user query.
2. Present **only that smaller subset** to the LLM.

---

### 🎯 Benefits of This Strategy

- ✅ **Improved Planning**: LLM focuses on fewer, more relevant choices.
- ✅ **Lower Costs**: Commercial LLM pricing is often based on **prompt + output token length**.
- ✅ **Reduced Prompt Complexity**: Easier to interpret and debug.

---

### 🕒 Tradeoff: Additional Latency

- ❗ This RAG pre-selection step **adds overhead** to your application's runtime.
- 📉 Use it **only when** you notice degraded performance due to too many tools.

---

Let’s now walk through how to implement this tool-reduction strategy using a RAG component.

In [5]:
import ast
from typing import Annotated, TypedDict

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_core.vectorstores.in_memory import InMemoryVectorStore
from langchain_openai import AzureChatOpenAI
from langchain_ollama import OllamaEmbeddings

from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

@tool
def calculator(query: str) -> str:
    """A simple calculator tool. Input should be a mathematical expression."""
    return ast.literal_eval(query)

search = DuckDuckGoSearchRun()
tools = [search, calculator]

embeddings = OllamaEmbeddings(model="mxbai-embed-large")
model = AzureChatOpenAI(azure_deployment="gpt-4o",
                        api_version="2024-10-21",
                        model="gpt-4o" #for tracing
                        )
tools_retriever = InMemoryVectorStore.from_documents(
    [Document(tool.description, metadata={"name": tool.name}) for tool in tools],
    embeddings
).as_retriever()

class State(TypedDict):
    messages: Annotated[list, add_messages]
    selected_tools: list

def model_node(state: State) -> State:
    selected_tools = [
        tool for tool in tools if tool.name in state["selected_tools"]
    ]
    res = model.bind_tools(selected_tools).invoke(state["messages"])
    return {"messages":res}

def select_tools(state: State) -> State:
    query = state["messages"][-1].content
    tool_docs = tools_retriever.invoke(query)
    return {"selected_tools": [doc.metadata["name"] for doc in tool_docs]}

builder = StateGraph(State)
builder.add_node("select_tools", select_tools)
builder.add_node("model", model_node)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "select_tools")
builder.add_edge("select_tools", "model")
builder.add_conditional_edges("model",tools_condition)
builder.add_edge("tools", "model")

graph = builder.compile()

In [6]:
graph_to_image(graph,"graph_rag_for_tool_choosing.png")

'File graph_rag_for_tool_choosing.png already exists. Skipping image generation.'

The visual representation can be seen here:

![Calling tool first](../img/graph_rag_for_tool_choosing.png)

> ### 🔎 Note
>
> This approach is **very similar** to the standard agent architecture.
>
> The **only difference** is the inclusion of a `select_tools` node **before** entering the agent loop.
>
> - The `select_tools` node performs a **RAG-based tool selection** step.
> - After this, the flow continues exactly as in the **regular agent architecture**:
>   - LLM decides what to do.
>   - Tools are executed.
>   - Loop continues until the stop condition is met.


In [7]:
input = {
  "messages": [
    HumanMessage("""How old was the 30th president of the United States when 
        he died?""")
  ]
}
for c in graph.stream(input):
    print(c)

{'select_tools': {'selected_tools': ['duckduckgo_search', 'calculator']}}
{'model': {'messages': AIMessage(content='The 30th president of the United States was Calvin Coolidge. He was born on July 4, 1872, and he died on January 5, 1933. Calvin Coolidge was **60 years old** when he passed away.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 116, 'total_tokens': 170, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BbtKljc6YnnjFpxmdpxR52RkpLCBW', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'sever

### Observation

The first step in this modified agent architecture is querying the retriever  
to get the most relevant tools for the current user query.

Once the subset of relevant tools is selected, the flow proceeds into  
the standard agent architecture, where:

- The LLM selects actions based on the current context and available tools.
- The selected tools are executed.
- The loop continues until the stop condition is met.


Let's go back to the main [file](../README.md/#more-agent-architectures).